In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
#
url = "https://raw.githubusercontent.com/ddw26/Team-Vand/circle_role_crkaide/output_data/df_values.csv"
catch_df = pd.read_csv(url)
catch_df.head(5)

,Date file created,Month of intercept,Time of intercept,Data sampling period,Sub region of trip,Kind of day,Fish target species,Mode of fishing,Collapsed area of fishing,Distance from shore,Hours on boat,Hours fished,Catch fish,Number of fish avail for inspection
0,2020-03-16,2,1512,January/February,South Atlantic (NC; SC; GA; EFL),Weekday,SPOTTED SEATROUT,Charter boat,Inland,not applicable,7.0,6.0,Yes,3
1,2020-03-16,1,1250,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,Charter boat,Inland,not applicable,5.0,4.5,No,0
2,2020-03-16,1,1250,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,Charter boat,Inland,not applicable,5.0,4.5,No,0
3,2020-03-16,2,1303,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,Charter boat,Inland,not applicable,6.0,4.0,No,0
4,2020-03-16,2,1303,January/February,South Atlantic (NC; SC; GA; EFL),Weekend,RED DRUM,Charter boat,Inland,not applicable,6.0,4.0,No,0


# Split the Data into Training and Testing

In [4]:
catch_df = catch_df.drop('Date file created', axis =1)
catch_df = catch_df.drop('Number of fish avail for inspection', axis=1)

In [5]:
# Create our features
X = catch_df.drop('Catch fish', axis=1)
X = pd.get_dummies(X)

# Create our target
#y = catch_df.drop('Fish target species', axis=1)
y = catch_df['Catch fish']

In [6]:
X.describe()

,Month of intercept,Time of intercept,Hours on boat,Hours fished,Data sampling period_January/February,Data sampling period_July/August,Data sampling period_March/April,Data sampling period_May/June,Data sampling period_November/December,Data sampling period_September/October,...,Fish target species_TAUTOG,Mode of fishing_Charter boat,Mode of fishing_Head boat,Mode of fishing_Private/Rental boat,Collapsed area of fishing_Inland,Collapsed area of fishing_Ocean <= 3 mi (all but WFL),Collapsed area of fishing_Ocean > 3 mi (all but WFL),Distance from shore_<= 3 miles (all but WFL),Distance from shore_> 3 miles (all but in WFL),Distance from shore_not applicable
count,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,...,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000,15276.000000
mean,7.599699,1297.932247,6.509525,4.251702,0.005826,0.337523,0.035742,0.293074,0.111678,0.216156,...,0.078555,0.222833,0.335494,0.441673,0.439251,0.291568,0.269180,0.291568,0.269180,0.439251
std,2.088315,313.215859,2.315059,1.649068,0.076109,0.472880,0.185653,0.455187,0.314981,0.411635,...,0.269051,0.416161,0.472178,0.496603,0.496312,0.454500,0.443548,0.454500,0.443548,0.496312
min,1.000000,41.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,1150.000000,5.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,1345.000000,6.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,1520.000000,8.000000,5.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,12.000000,2358.000000,24.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
y.value_counts()

Yes    8380
No     6896
Name: Catch fish, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [9]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier (n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [10]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8294176288207089

In [11]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1444,  268],
       [ 389, 1718]])

In [12]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         No       0.79      0.84      0.82      0.81      0.83      0.69      1712
        Yes       0.87      0.82      0.84      0.84      0.83      0.69      2107

avg / total       0.83      0.83      0.83      0.83      0.83      0.69      3819



In [13]:
# List the features sorted in descending order by feature importance
#drop Numbe of fish
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.2828466070986044, 'Time of intercept'),
 (0.11756425742051739, 'Hours fished'),
 (0.1140799626010934, 'Hours on boat'),
 (0.0762080392637313, 'Mode of fishing_Charter boat'),
 (0.06120085490636556, 'Mode of fishing_Head boat'),
 (0.05920632103330721, 'Month of intercept'),
 (0.023621896833260336,
  'Sub region of trip_North Atlantic (ME; NH; MA; RI; CT)'),
 (0.02329483895500523, 'Fish target species_STRIPED BASS'),
 (0.02164161270694076, 'Mode of fishing_Private/Rental boat'),
 (0.018953593982443954, 'Sub region of trip_South Atlantic (NC; SC; GA; EFL)'),
 (0.01657736536993581, 'Fish target species_SUMMER FLOUNDER'),
 (0.014565123818550883,
  'Sub region of trip_Mid-Atlantic (NY; NJ; DE; MD; VA)'),
 (0.014014098039738368, 'Kind of day_Weekday'),
 (0.012569461296222433, 'Fish target species_BLACK SEA BASS'),
 (0.012456268803745597, 'Kind of day_Weekend'),
 (0.012132105738581304, 'Data sampling period_July/August'),
 (0.011389041407460564, 'Collapsed area of fishing_Inland'),
 (0.009

### Easy Ensemble AdaBoost Classifier

In [14]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [15]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7368713378635523

In [16]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1223,  489],
       [ 507, 1600]])

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         No       0.71      0.71      0.76      0.71      0.74      0.54      1712
        Yes       0.77      0.76      0.71      0.76      0.74      0.54      2107

avg / total       0.74      0.74      0.73      0.74      0.74      0.54      3819

